In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:22pt;}
div.text_cell_render.rendered_html{font-size:18pt;}
div.text_cell_render ul li pre{font-size:22pt; line-height:30px;}
div.output {font-size:22pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:22pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:22pt;padding:5px;}
table.dataframe{font-size:22px;}
</style>
"""))

<b><font size="7" color="red">ch14. 웹데이터 수집 - 동적</font></b>
# 1절. Selenium을 이용한 동적 웹크롤링 문법
- Selenium docs : https://selenium-python.readthedocs.io/
`pip install selenium`
- import에서 에러날 경우
`pip install requests`나 `conda install urllib3==1.26.18`

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

In [10]:
driver = webdriver.Chrome()
driver.get("http://www.python.org")
elem = driver.find_element(By.NAME, 'q')
elem.clear()
elem.send_keys('pycon')

In [3]:
from selenium.webdriver.common.keys import Keys
elem.send_keys(Keys.RETURN) # Enter를 입력

In [4]:
btn_elem = driver.find_element(By.CSS_SELECTOR, 'button#submit')
btn_elem.click()

In [ ]:
result_list = driver.find_elements(By.CSS_SELECTOR, 'li > h3 > a')
for result in result_list:
    title = result.text
    link  = result.get_attribute('href')
    print('{} - {}'.format(title, link))

In [6]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser')
result_list = soup.select('li > h3 > a')
for result in result_list:
    print("{} - {}".format(result.text, result.attrs['href']))

PSF PyCon Trademark Usage Policy - /psf/trademarks/pycon
PyCon Italia 2016 (PyCon Sette) - /events/python-events/378/
PyCon Australia 2013 - /events/python-events/57/
PyCon AU 2019 - /events/python-events/776/
PyCon NL 2025 - /events/python-events/2084/
PyCon Australia 2014 - /events/python-events/10/
PyCon Ireland 2012 - /events/python-events/76/
PyCon Ireland 2016 - /events/python-events/429/
PyCon Ireland 2022 - /events/python-events/1320/
PyCon Australia 2014 - /events/python-events/1447/
PyCon Ireland 2024 - /events/python-events/1862/
PyCon APAC 2025 - /events/python-events/1879/
PyCon AU 2018 - /events/python-events/696/
PyCon APAC 2022 - /events/python-events/1216/
PyCon PH 2024 - /events/python-events/1661/
PyCon Ireland 2023 - /events/python-events/1568/
PyCon PL 2014 - /events/python-events/191/
PyCon MY 2015 - /events/python-events/313/
PyCon Ireland 2015 - /events/python-events/333/
PyCon AU 2015 - /events/python-events/273/


In [7]:
from urllib.parse import urlparse
current_url =driver.current_url # 셀레니윰을 통해 접근한 url
print(current_url)
parse_url = urlparse(current_url)
print(parse_url)
domain = f'{parse_url.scheme}://{parse_url.netloc}'
print(domain)

https://www.python.org/search/?q=pycon&submit=
ParseResult(scheme='https', netloc='www.python.org', path='/search/', params='', query='q=pycon&submit=', fragment='')
https://www.python.org


In [8]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
result_list = soup.select('li > h3 > a')
for result in result_list:
    print("{} - {}".format(result.text, domain+result.attrs['href']))

PSF PyCon Trademark Usage Policy - https://www.python.org/psf/trademarks/pycon
PyCon Italia 2016 (PyCon Sette) - https://www.python.org/events/python-events/378/
PyCon Australia 2013 - https://www.python.org/events/python-events/57/
PyCon AU 2019 - https://www.python.org/events/python-events/776/
PyCon NL 2025 - https://www.python.org/events/python-events/2084/
PyCon Australia 2014 - https://www.python.org/events/python-events/10/
PyCon Ireland 2012 - https://www.python.org/events/python-events/76/
PyCon Ireland 2016 - https://www.python.org/events/python-events/429/
PyCon Ireland 2022 - https://www.python.org/events/python-events/1320/
PyCon Australia 2014 - https://www.python.org/events/python-events/1447/
PyCon Ireland 2024 - https://www.python.org/events/python-events/1862/
PyCon APAC 2025 - https://www.python.org/events/python-events/1879/
PyCon AU 2018 - https://www.python.org/events/python-events/696/
PyCon APAC 2022 - https://www.python.org/events/python-events/1216/
PyCon PH 2

In [11]:
driver.close() # 브라우저 종료

# 2절. 동적웹크롤링 예제
## 2.1 다음뉴스검색

In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
driver = webdriver.Chrome()
driver.get('https://www.daum.net/')
# driver.implicitly_wait(0.5) # 최대로 0.5초동안 대기
time.sleep(0.5) # 0.5초 멈춤
driver.find_element(By.CLASS_NAME, 'tf_keyword').click()
query = input('검색할 단어는?')
driver.find_element(By.CSS_SELECTOR, 'input[type="text"]').send_keys(query)
driver.find_element(By.CLASS_NAME, 'btn_ksearch').click()

검색할 단어는?비트코인


In [15]:
# 뉴스 탭 클릭
# driver.find_elements(By.CSS_SELECTOR, 'ul.list_tab > li')[1].click()
driver.find_element(By.LINK_TEXT, '뉴스').click()

In [20]:
# 뉴스 제목과 link를 list 추가
news_list = []
strongs = driver.find_elements(By.CSS_SELECTOR, 'div.item-title > strong')
# print(len(strongs))
for strong in strongs:
    a = strong.find_element(By.TAG_NAME, 'a')
    title = a.text
    link = a.get_attribute('href')
    news_list.append([title, link])    

In [24]:
# 다음페이지로
page_nav = driver.find_element(By.CSS_SELECTOR, 'div.inner_paging')
print(page_nav.text)
next_page = page_nav.find_element(By.LINK_TEXT, "2")
next_page.click()

1 2 3 4 5


In [25]:
driver.close()

## 2.2 페이징 처리
- 다음 뉴스 페이징 처리 : 원하는 keyword를 원하는 페이지만큼 추출

In [27]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
driver = webdriver.Chrome()
driver.get('https://www.daum.net/')
# driver.implicitly_wait(0.5) # 최대로 0.5초동안 대기
time.sleep(0.5) # 0.5초 멈춤
driver.find_element(By.CLASS_NAME, 'tf_keyword').click()
query = input('검색할 단어는?')
driver.find_element(By.CSS_SELECTOR, 'input[type="text"]').send_keys(query)
driver.find_element(By.CLASS_NAME, 'btn_ksearch').click()
time.sleep(2)
# 뉴스탭 이동
driver.find_element(By.LINK_TEXT, '뉴스').click()

# 뉴스 제목과 link를 list 추가
news_list = []
pages = 3 # int(input('몇 페이지 크롤링할까?'))
for page in range(1, pages+1):
    strongs = driver.find_elements(By.CSS_SELECTOR, 'div.item-title > strong')
    # print(len(strongs))
    for strong in strongs:
        a = strong.find_element(By.TAG_NAME, 'a')
        title = a.text
        link = a.get_attribute('href')
        news_list.append([title, link])   
    page_nav = driver.find_element(By.CSS_SELECTOR, 'div.inner_paging')
    next_page = page_nav.find_element(By.LINK_TEXT, str(page+1))
    next_page.click()
    time.sleep(2)
driver.close()
import pandas as pd
pd.DataFrame(news_list, columns=['title','link']).tail(3)

검색할 단어는?비트코인


,title,link
27,"비트코인 ETF 막힌 한국…서학개미, 미국 레버리지로 '우회 매수' 급증",http://v.daum.net/v/20251111140546725
28,불장에 코인 거래대금은 연초 대비 7분의 1 뚝… 거래소 ‘1억 상금’ 유치전까지,http://v.daum.net/v/20251112154826414
29,압수된 비트코인 9조 자금세탁…40대 중국인 징역형,http://v.daum.net/v/20251112072049232


## 2.3 맞춤법 검사기
- 네이버 맞춤법 검사기 이용

In [35]:
from bs4 import BeautifulSoup

In [36]:
driver = webdriver.Chrome()
driver.get('https://www.naver.com/')
time.sleep(1)
elem = driver.find_element(By.ID, 'query')
elem.send_keys('맞춤법 검사기')
elem.send_keys(Keys.RETURN) # form태그의 submit 클릭 효과

textarea = driver.find_element(By.CLASS_NAME, 'txt_gray')
textarea.clear()
textarea.send_keys('안뇽하세요. 반갑숩니다. 감샤합니다')
btn = driver.find_element(By.CLASS_NAME, 'btn_check')
btn.click()
time.sleep(2)

soup = BeautifulSoup(driver.page_source, 'html.parser')
result = soup.select_one('p._result_text.stand_txt').text
print(result)
driver.close()

안녕하세요. 반갑습니다. 감사합니다


### 맞춤법 검사 글자수
- 맞춤법검사전.txt 읽어 300자 단위로 짤라 
- list에 넣어 동적 웹크롤링을 한다.
- 결과출력

In [11]:
with open('data/ch14_맞춤법검사전.txt', 'r', encoding='utf-8') as f:
    text = f.read()
ready_text = []   # 300자씩 나눠진 text
while(len(text) > 300):
    temp = text[:300]
    last_dot_index = temp.rfind('\n')
    ready_text.append(text[:last_dot_index])
    text = text[last_dot_index+1 : ]
ready_text.append(text)
print(ready_text)
print([len(ready) for ready in ready_text])

['동네아이들 볏짚 깔아주기는 이미 2주전에 마쳤는데, \n\n정작 우리집 동물 가족들의 겨울준비가 늦었습니다.\n\n며칠전부터 우리집 닭장과 고양이들 겨울준비 해 주느라 완전 노가다로 힘든일 마쳤습니다.\n\n고양이들 집에 전기공사는 다시 해야하는데 전기박사 큰아들이 미국가버려 고민중입니다.\n\n일단 빨아두엇던 담요와 패드들 집안 청소 다 하고 폭신하게 깔아주니 \n\n따듯하고 깨끗한걸 어찌 그렇게 잘 아는지 바로 그날로 새 자리로 들어가 자는군요.\n\n\u200b\n\n어제는 닭장 대 청소를 했습니다. 쥐구멍들 막느라고 건재상에서 우레탄 폼을 사다가 \n', '하늘이 보이도록 흙을 다 파버려 그건 돌이나 비닐족같은걸로는 소용이 업었습니다.\n\n그런데 전문가들이나 할 일을 내가 해 보겠다고 재료사다가 깨끗이 주면을 바람기계로 먼지 날린다음 \n\n우레탄을 쏴주는데 내 생각대로 말을 안듣는군요.ㅠㅠ\n\n정말 전문가가 보면 웃을일입니다만 일단 큰 바람구멍들은 막아서 혼자 웃었습니다.^^\n\n\u200b\n\n창문까지 물수건으로 먼지나마 닦아내니 햇볓이라도 좀더 잘 들기를 바랍니다.\n\n어제 오후 내내 막노동을 하고낫더니 전날 코로나와 독감주사가지 맞은 뒤라 몸살처럼 아팠습니다.\n', '다음주엔 다시 미국여행을 가게 되어 할일이 태산이군요.\n\n이제 닭장의 방풍준비는 다녀와서 해야겠습니다.\n\n아직은 난로는 안켜주고 밤엔 온열등만 켜줍니다. \n\n구오가 죽는줄 알았는데 6일만에 처음으로 음식을 아주 조금, \n\n조금이라기보다 밥알 10알정도 크기의 고기를 억지로 입을 벌리고 넣어줬더니 삼켰습니다.\n\n그리고 저녁엔 맛있는 닭고기와 닭간으로 만든 고양이 캔을 섞어서 사료를 20알 정도를 먹었답니다.\n\n아! 얼마나 기쁘던지요.\n\n오늘아침 일주일만에 드디어 사료에 어제처럼 고양이 캔에 비벼서 어제보다 조금 많이 먹었습니다.\n', '밥만 먹으면 살아난겁니다.\n\n\u200b\n\n원장님 왕진와주셔서 채혈한것 검사결과는 너무나 다행히 아주 

In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup
driver = webdriver.Chrome()
time.sleep(1)
driver.get('https://www.naver.com/')
time.sleep(0.5)
elem = driver.find_element(By.ID, 'query')
elem.send_keys('맞춤법 검사기')
elem.send_keys(Keys.RETURN) # form태그의 submit 클릭 효과

textarea = driver.find_element(By.CLASS_NAME, 'txt_gray')
results = '' # 맞춤법 검사 완료된 text
for i, ready in enumerate(ready_text):
    print(f'검사중...{i+1}/{len(ready_text)}')
    #textarea.clear()
    textarea.send_keys(Keys.CONTROL, 'a') # ctrl+a
    textarea.send_keys(ready)
    time.sleep(1)
    btn = driver.find_element(By.CLASS_NAME, 'btn_check')
    btn.click()
    time.sleep(2)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    result = soup.select_one('p._result_text.stand_txt').text
    results += result + '\n'
    time.sleep(1)
driver.close()

검사중...1/6
검사중...2/6
검사중...3/6
검사중...4/6
검사중...5/6
검사중...6/6


In [17]:
results

'동네 아이들 볏짚 깔아주기는 이미 2주 전에 마쳤는데, 정작 우리 집 동물 가족들의 겨울 준비가 늦었습니다.며칠 전부터 우리 집 닭장과 고양이들 겨울 준비해 주느라 완전 막노동으로 힘든 일 마쳤습니다.고양이들 집에 전기공사는 다시 해야 하는데 전기 박사 큰아들이 미국 가버려 고민 중입니다.일단 빨아두었던 담요와 패드들 집안 청소 다 하고 폭신하게 깔아주니 따듯하고 깨끗한 걸 어찌 그렇게 잘 아는지 바로 그날로 새 자리로 들어가 자는군요.\u200b어제는 닭장 대 청소를 했습니다. 쥐구멍들 막느라고 건재상에서 우레탄 폼을 사다가\n하늘이 보이도록 흙을 다 파 버려 그건 돌이나 비닐족같은걸로는 소용이 업었습니다.그런데 전문가들이나 할 일을 내가 해 보겠다고 재료 사다가 깨끗이 주면을 바람 기계로 먼지 날린 다음 우레탄을 쏴주는데 내 생각대로 말을 안듣는군요.ㅠㅠ정말 전문가가 보면 웃을 일입니다만 일단 큰 바람구멍들은 막아서 혼자 웃었습니다.^^\u200b창문까지 물수건으로 먼지나마 닦아내니 햇볕이라도 좀 더 잘 들기를 바랍니다.어제 오후 내내 막노동을 하고 났더니 전날 코로나와 독감주사 가지 맞은 뒤라 몸살처럼 아팠습니다.\n다음 주엔 다시 미국 여행을 가게 되어 할 일이 태산이군요.이제 닭장의 방풍 준비는 다녀와서 해야겠습니다.아직은 난로는 안 켜주고 밤엔 온열등만 켜줍니다. 구오가 죽는 줄 알았는데 6일 만에 처음으로 음식을 아주 조금, 조금이라기보다 밥알 10알 정도 크기의 고기를 억지로 입을 벌리고 넣어줬더니 삼켰습니다.그리고 저녁엔 맛있는 닭고기와 달간으로 만든 고양이 캔을 섞어서 사료를 20알 정도를 먹었답니다.아! 얼마나 기쁘던지요.오늘 아침 일주일 만에 드디어 사료에 어제처럼 고양이 캔에 비벼서 어제보다 조금 많이 먹었습니다.\n밥만 먹으면 살아난 겁니다.\u200b원장님 왕진 와주셔서 채혈한 것 검사 결과는 너무나 다행히 아주 정상이랍니다.그러니까 노령이라 달팽이관이나 신경계 이상이 온 것 같습니다.아직 고개는 왼쪽으로 살짝 돌아간 상태지만 걸음